In [14]:
import numpy as np
import torch
import torch.nn as nn
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm

#constants
TARGET = np.array([[-0.1, 0.2, 0.3]], dtype = np.double)


EPS = 0.5

W0 = np.array([[-0.5, -0.03, -0.08],
               [ 0.15,  0.19,  0.27]], dtype = np.double)
B0 = np.array([-0.46, -0.02], dtype = np.double)

In [20]:
#a very simple neural net
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc0 = nn.Linear(3, 2)
        self.fc0.weight = torch.nn.Parameter(torch.from_numpy(W0))
        self.fc0.bias = torch.nn.Parameter(torch.from_numpy(B0))
        
    def forward(self, x):
        return self.fc0(x)
        
#Compute bound using Interval Bound Propagation, using auto_LiRPA API
target = torch.from_numpy(TARGET)
print(target.shape)
ball = PerturbationLpNorm(norm=np.inf, eps=EPS)
ball_tensor = BoundedTensor(target, ball)
print(ball_tensor)

original_model = LinearModel()

lirpa_model = BoundedModule(original_model, torch.empty_like(target))

print(lirpa_model(ball_tensor))

# lb, ub = lirpa_model.compute_bounds(IBP=True, method = 'forward')
lb, ub = lirpa_model.compute_bounds(IBP=True, method = 'backward')

torch.Size([1, 3])
<BoundedTensor: tensor([[-0.1000,  0.2000,  0.3000]], dtype=torch.float64), PerturbationLpNorm(norm=inf, eps=0.5)>
tensor([[-0.4400,  0.0840]], dtype=torch.float64, grad_fn=<AddBackward0>)


In [ ]:
#Compute the output of the network
def forward(x):
    fc0 = np.matmul(x, W0.transpose()) + B0
    return fc0
print(lirpa_model(ball_tensor).detach().numpy())
print(forward(TARGET))
assert(np.array_equal(forward(TARGET),
                      lirpa_model(ball_tensor).detach().numpy()))

In [21]:
##Compute bound using Interval Bound Propagation, using my own implementation.
##The closed-form solution is in eq(6), https://arxiv.org/pdf/1810.12715.pdf
my_ball = np.array([TARGET - EPS, TARGET + EPS])
print(my_ball.shape)

def my_IBP(prev_bound: np.array, W: np.array, b: np.array):
    """
    prev_bound: 2x784 prev_bound[0][i]: lower of unit ith, prev_bound[1]i]: upper of unit ith
    W: 784x256
    b: 256
    """
    assert(prev_bound.shape[0]==2)
    assert(prev_bound.shape[1]==W.shape[0])
    assert(W.shape[1] == b.shape[0])
    
    prev_u = (prev_bound[0,:] + prev_bound[1,:])/2

    prev_r = (prev_bound[1,:] - prev_bound[0,:])/2
    
    u = np.matmul(prev_u , W) + b
    
    r = np.matmul(prev_r , abs(W) )
    
    new_bound_lower = u - r
    
    new_bound_upper = u + r
    
    new_bound = np.array([new_bound_lower, new_bound_upper])
    
    return new_bound

my_bound = my_IBP(my_ball, W0.transpose(), B0)
lirpa_bound = np.array([lb.detach().numpy(), ub.detach().numpy()])

print(my_bound)
print(lirpa_bound)

#cannot use array_equal due to some floating point difference?
assert(np.allclose(my_bound, 
                   lirpa_bound))


(2, 1, 3)


AssertionError: 

In [18]:
## Computer bound using CROWN
print(lirpa_model(ball_tensor))
crown_lb, crown_ub = lirpa_model.compute_bounds(IBP=False, method = 'backward')
crown_bound = np.array([crown_lb.detach().numpy(), crown_ub.detach().numpy()])
print(crown_bound)

def compute_and_compare_bounds(original_model, eps, norm, IBP, method):
    global TARGET
    input_data = torch.from_numpy(TARGET)
    model = BoundedModule(original_model, torch.empty_like(input_data))
    ptb = PerturbationLpNorm(norm=norm, eps=eps)
    ptb_data = BoundedTensor(input_data, ptb)
    pred = model(ptb_data)
#     label = torch.argmax(pred, dim=1).cpu().detach().numpy()
    # Compute bounds.
    lb, ub = model.compute_bounds(IBP=IBP, method=method)
    # Compute dual norm.
    if norm == 1:
        q = np.inf
    elif norm == np.inf:
        q = 1.0
    else:
        q = 1.0 / (1.0 - (1.0 / norm))
    # Compute reference manually.
    weight, bias = list(model.parameters())
    norm = weight.norm(p=q, dim=1)
    expected_pred = input_data.matmul(weight.t()) + bias
    expected_ub = eps * norm + expected_pred
    expected_lb = -eps * norm + expected_pred

    # Check equivalence.
#     self.assertEqual(expected_pred, pred)
#     self.assertEqual(expected_ub, ub)
#     self.assertEqual(expected_lb, lb)
    
compute_and_compare_bounds(original_model, eps=0.3, norm=np.inf, IBP=False, method='backward')


tensor([[-0.4400,  0.0840]], dtype=torch.float64, grad_fn=<AddBackward0>)
[[[-0.745 -0.221]]

 [[-0.135  0.389]]]
